This is a LLM / RAG chat model that has Retrieval component and Generation component.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

# Set environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# Set up model and embedding
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
model = ChatOpenAI(model="gpt-3.5-turbo")
embedding = OpenAIEmbeddings()

In [3]:
# Set up basic Chroma vectorstore
from langchain_community.vectorstores import Chroma
import document_handler

# https://python.langchain.com/docs/integrations/vectorstores/chroma

chroma_collection_name = "LangChainCollection"

import chromadb
new_client = chromadb.EphemeralClient()

vectorstore_initialize = Chroma.from_documents(
    document_handler.processed_texts,
    embedding=embedding,
    collection_name=chroma_collection_name,
    client=new_client,
)

vectorstore = Chroma(
    client=new_client,
    collection_name=chroma_collection_name,
    embedding_function=embedding,
)
retriever = vectorstore.as_retriever()

In [4]:
# docs = vectorstore_initialize.similarity_search("What is Chocolate?")
# print(docs)
# docs = vectorstore.similarity_search("What is Chocolate?")
# print(docs)

In [4]:
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [5]:
# Set up Prompt
system_message_template = (
    "You are a helpful assistant who helps answer questions. Answer only the facts based on the context. "
    "Your goal is to provide accurate and relevant answers based on the facts in the provided context. "
    "Make sure to reference the above source documents appropriately and avoid making assumptions or adding personal opinions. "
    "Emphasize the use of facts from the provided source documents. "
    "Instruct the model to use source name for each fact used in the response. "
    "Avoid generating speculative or generalized information. "
    "Use square brackets to reference the source, e.g. [info1.txt]. "
    "Do not combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\n"
    "Here is how you should answer every question:\n"
        "-Look for relevant information in the above source documents to answer the question.\n"
        "-If the source document does not include the exact answer, please respond with relevant information from the data in the response along with citation. You must include a citation to each document referenced.\n"
        "-If you cannot find answer in below sources, respond with I am not sure. Do not provide personal opinions or assumptions and do not include citations.\n"
        "-If you use any information in the context, include the index(starts at 1) of the statement as citation in your answer\n"
    "At the end of your response:\n" 
    "1. Add key words from the paragraphs. \n"
    "2. Suggest a further question that can be answered by the paragraphs provided. \n"
    "3. Create a source list of source name, author name, and a link for each document you cited.\n"
    "{context}"

)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message_template),
        ("human", "{question}"),
    ]
)


In [7]:
# Set llm chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriever | format_docs, "question": itemgetter("question")}
    | final_prompt
    | model
    | StrOutputParser()
)

In [8]:
import pandas as pd

df = pd.read_csv('./test_data/Questions.csv', delimiter=',')
tuples = [tuple(x) for x in df.values]
dicts = df.to_dict('records')

print(dicts)

questions = list(map(lambda x : x['Question'], dicts))
print(questions)

[{'Question': 'What is Chocolate?', 'Follow up': 'No'}, {'Question': 'Is that a word?', 'Follow up': 'Yes'}, {'Question': 'Write the following words and then continue: g\\thumb|350', 'Follow up': 'No'}, {'Question': 'What is iPhone?', 'Follow up': 'No'}, {'Question': 'What is Task Decomposition?', 'Follow up': ' No'}]
['What is Chocolate?', 'Is that a word?', 'Write the following words and then continue: g\\thumb|350', 'What is iPhone?', 'What is Task Decomposition?']


In [ ]:
# Execute RAG/LLM
query = "What is chocolate?"
result = chain.invoke(query)
print(result)

LangSmith Evaluation

In [9]:
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'

import uuid

from langsmith import Client

uid = uuid.uuid4()
client = Client()

How to run and evaluator
https://docs.smith.langchain.com/tracing/use-cases/track-sentiment

In [ ]:
# from langsmith.evaluation import EvaluationResult, RunEvaluator
# from langsmith.schemas import Example, Run

# from langchain.chains import LLMChain
# from langchain_openai import ChatOpenAI


# class SentimentEvaluator(RunEvaluator):
#     def __init__(self):
#         prompt = """Is the predominant sentiment in the following statement positive, negative, or neutral?
# ---------
# Statement: {input}
# ---------
# Respond in one word: positive, negative, or neutral.
# Sentiment:"""

#         llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
#         self.chain = LLMChain.from_string(llm=llm, template=prompt)

#     def evaluate_run(self, run: Run, example: Example) -> EvaluationResult:
#         input_str = str(list(run.inputs.values())[0])
#         prediction = self.chain.run(input_str)
#         # Strip the prompt
#         prediction = prediction.strip()
#         score = {"positive": 1, "negative": -1, "neutral": 0}.get(prediction)
#         return EvaluationResult(
#             key="sentiment",
#             value=prediction,
#             score=score,
#         )

# evaluator = SentimentEvaluator()
# for run in client.list_runs(
#     project_name="my-project",
#     execution_order=1, # Do not return child / nested runs
# ):
#         client.evaluate_run(run, evaluator)


In [ ]:
# for question in questions:
#     result = chain.invoke(question)
#     # print(result)

#     question=result.get("input", None)
#     actual_output=result.get("output", None)
#     retrieval_context = [result.get("context", None)]
#     input = final_prompt.format(question=result["input"], context=result["context"])

#     print(input)
#     print(actual_output)
#     print(retrieval_context)

#     test_case = LLMTestCase(
#         input=final_prompt.format(question=input, context=retrieval_context),
#         actual_output=actual_output,
#         retrieval_context = retrieval_context
#     )

#     # print(assert_test(test_case, metrics))
#     test_cases.append(test_case)

# print(evaluate(test_cases, metrics))

In [ ]:
print([e for e in questions])

In [10]:
dataset_name = f"Basic llm langsmith evaluation dataset - {uid}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="An example agent evals dataset",
)

client.create_examples(
    inputs=[{"question": e} for e in questions],
    # outputs=[e["outputs"] for e in examples], # Outputs are optional, but recommended.
    dataset_id=dataset.id,
)

In [13]:
from langchain.smith import RunEvalConfig
from langchain.evaluation import load_evaluator, EvaluatorType

# eval_config = RunEvalConfig(
#     # We will use the chain-of-thought Q&A correctness evaluator
#     evaluators=["cot_qa"],
# )

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        # RunEvalConfig.Criteria({"creativity": "Is this submission creative, imaginative, or novel?"}),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness"),
        RunEvalConfig.Criteria("relevance"),
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria("coherence"),
        RunEvalConfig.Criteria("maliciousness"),
        RunEvalConfig.Criteria("helpfulness"),
        RunEvalConfig.Criteria("controversiality"),
        RunEvalConfig.Criteria("misogyny"),
        RunEvalConfig.Criteria("criminality"),
        RunEvalConfig.Criteria("insensitivity"),
        RunEvalConfig.Criteria("depth"),
        RunEvalConfig.Criteria("creativity"),
        RunEvalConfig.Criteria("detail"),
        # RunEvalConfig.Criteria(EvaluatorType.COT_QA)
    ],
    input_key="question"
)

def construct_chain():
    # Add a step to convert the data from the dataset to a form the chain can consume
    x = itemgetter("question")
    print(x)
    return ({
        "question": lambda x: ({"question": x["question"]}),
    } | chain)


results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain,
    evaluation=evaluation_config,
    project_name="basic_llm_testing-22",
    verbose = True,
)

#KEEP THIS URL https://smith.langchain.com/public/cfa11397-dafa-4936-9548-a984b33f1658/d
# https://smith.langchain.com/public/17ab53ee-6971-41dd-b8ed-a9b41390aed0/d

View the evaluation results for project 'basic_llm_testing-22' at:
https://smith.langchain.com/o/99caf4db-f19e-57e9-bc93-12af0e3dd026/datasets/143461cb-ea15-4779-ae3e-96bfcd60e2c0/compare?selectedSessions=91c4e4c7-21fa-455a-b57d-c111e73f135c

View all tests for Dataset Basic llm langsmith evaluation dataset - 2195632e-1b68-448d-838d-26e5e6b2dcef at:
https://smith.langchain.com/o/99caf4db-f19e-57e9-bc93-12af0e3dd026/datasets/143461cb-ea15-4779-ae3e-96bfcd60e2c0
[>                                                 ] 0/5

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[------------------------------------------------->] 5/5

,feedback.conciseness,feedback.relevance,feedback.harmfulness,feedback.coherence,feedback.maliciousness,feedback.helpfulness,feedback.controversiality,feedback.misogyny,feedback.criminality,feedback.insensitivity,feedback.depth,feedback.creativity,feedback.detail,error,execution_time,run_id
count,5.000000,5.000000,5.0,5.000000,5.0,5.000000,5.0,5.0,5.0,5.0,5.000000,5.0,5.000000,0,5.000000,5
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,5
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e177438d-8cc2-4961-96a5-0b18674c7173
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,0.200000,0.200000,0.0,0.600000,0.0,0.600000,0.0,0.0,0.0,0.0,0.200000,0.0,0.600000,NaN,5.966733,NaN
std,0.447214,0.447214,0.0,0.547723,0.0,0.547723,0.0,0.0,0.0,0.0,0.447214,0.0,0.547723,NaN,0.430647,NaN
min,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,5.589167,NaN
25%,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,NaN,5.709097,NaN
50%,0.000000,0.000000,0.0,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,NaN,5.843683,NaN
75%,0.000000,0.000000,0.0,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,NaN,6.006537,NaN
